# 10.5 Extended source simulations

So far, we have only dealt with simulations for single or multiple point sources. In the next step of the tutorial
we give an example for simulating data from extended sources.


In [ ]:
import matplotlib.colors as colors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import tempfile

from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import astropy_mpl_style
from datetime import datetime
from funcs import run_comm
from subprocess import check_call, STDOUT
from xspec import Xset, Plot, AllData, ModelManager, Spectrum, Model, AllModels, Fit

plt.style.use(astropy_mpl_style)

In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
xmldir = f"{SIXTE}/share/sixte/instruments/athena-wfi/wfi_wo_filter_B4C"

xml0="{}/ld_wfi_ff_chip0.xml".format(xmldir)
xml1="{}/ld_wfi_ff_chip1.xml".format(xmldir)
xml2="{}/ld_wfi_ff_chip2.xml".format(xmldir)
xml3="{}/ld_wfi_ff_chip3.xml".format(xmldir)

## 10.5.1 Generating the SIMPUT file

Download Chandra images:

https://chandra.harvard.edu/photo/2014/etacar/fits/etaCar_xray_low.fits  (0.5-1.2 keV)  
https://chandra.harvard.edu/photo/2014/etacar/fits/etaCar_xray_med.fits  (1.2-2.0 keV)  
https://chandra.harvard.edu/photo/2014/etacar/fits/etaCar_xray_hi.fits   (2.0-7.0 keV)  

In [ ]:
# get source position form the image  etaCar_xray_low.fits
f = fits.open('etaCar_xray_hi.fits')
RAsrc = f[0].header['CRVAL1']
Decsrc = f[0].header['CRVAL2']
f.close()

# Use flux data from bibliography
flux_centralsrc_3_8 = 1E-11 # erg/cm2/s from Hamaguchi et al. 2014  -> assign it to the whole image
flux_diffemission_02_10 = 2.2E-12 # erg/cm2/s Seward et al. 2001

# Define bands and energt limits
E_minmax = {"high" : [2.,7.],
            "med"  : [1.2,2.],
            "low"  : [0.5,1.2]
           }
E_lowup = {"high" : [2.,10.],
           "med"  : [1.0,2.5],
           "low"  : [0.1,1.5]
          }

In [ ]:
# xspec file (simple powerlaw emission model with constant flux (Hamaguchi et al. 2014 does not give spectral parameters)

xcm = "constflux.xcm"

AllModels.clear()
AllData.clear()
mcmod = Model("power")
mcmod.powerlaw.PhoIndex = 1
mcmod.powerlaw.norm = 1

clobber = True
# If clobber is true and the file exists, it will be removed before creating a new one.
if os.path.exists(xcm):
    if clobber:
        os.remove(xcm)
    else:
        raise Exception(f"Xspec file ({xcm}) already exists: it will not be overwritten") 
Xset.save(xcm)

In [ ]:
# create simput file simput file 
f = fits.open('etaCar_xray_hi.fits')
RA = f[0].header['CRVAL1']
Dec = f[0].header['CRVAL2']
f.close()

sim_file_high = 'etaCar_high.fits'
comm = (f'simputfile Simput={sim_file_high} RA={RA} Dec={Dec} srcFlux={flux_centralsrc_3_8} ' + 
        f'Emin=3. Emax=8. Elow=2. Eup=10. XSPECFile={xcm} ImageFile=etaCar_xray_hi.fits clobber=yes')
#print(comm)
run_comm(comm, 'Creating simput file for eta car')

## 10.5.1 Simulating the observation

In [ ]:
# Running simulation
prefix = "high_"
RApoint = 161.56
DECpoint = -59.52
comm = (f'athenawfisim XMLFile0={xml0} XMLFile1={xml1} XMLFile2={xml2} XMLFile3={xml3} RA={RApoint} Dec={DECpoint} ' +
        f'Prefix={prefix} Simput={sim_file_high} Exposure=1000 clobber=yes')
#print(comm)
run_comm(comm,'Creating simput file for eta car')

In [ ]:
# Merging chips
evt_final_high = 'etaCar_high_chip_merged.fits'

comm = (f'ftmerge {prefix}chip0_evt.fits,{prefix}chip1_evt.fits,{prefix}chip2_evt.fits,{prefix}chip3_evt.fits ' + 
        f'{evt_final_high} clobber=yes')
run_comm(comm, 'Merging 4 event files')

In [ ]:
# Create the image of the merged evt file 
img_final_high = 'etaCar_high_merged_img.fits'
comm = (f'imgev EvtFile={evt_final_high} Image={img_final_high} CoordinateSystem=0 Projection=TAN NAXIS1=1063 ' +
        f'NAXIS2=1063 CUNIT1=deg CUNIT2=deg CRVAL1={RApoint} CRVAL2={DECpoint} CRPIX1=532 CRPIX2=532 ' + 
        f'CDELT1=-6.207043e-04 CDELT2=6.207043e-04 history=true clobber=yes')
#print(comm)
run_comm(comm, "Creating image from merged evt file")

In [ ]:
image_data = fits.getdata(img_final_high, ext=0)
hdu = fits.open(img_final_high)[0]
wcs = WCS(hdu.header)

fig = plt.figure(figsize=(14,8))
fig.suptitle("SImulation of the CDFS (5 ks)")
#cmap = plt.cm.winter
cmap = plt.cm.Blues_r

#plot image
ax = fig.add_subplot(1, 1, 1, projection=wcs)
im = ax.imshow(image_data, cmap=cmap, norm=colors.LogNorm(vmin=1e-5, vmax=np.max(image_data), clip=True), origin="lower")
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
fig.colorbar(im,ax=ax, pad=0.1)
plt.tight_layout();

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Generate a spectrum of chip0 and verify that indeed only photons between 2 and 10 keV are generated. Where do the photons detected below 2 keV come from?

In [ ]:
# Create spectrum
spec_file = f'{prefix}chip0_spec.pha'
comm = (f"makespec EvtFile={prefix}chip0_evt.fits Spectrum={spec_file} clobber=yes RSPPath={xmldir}")
#print(comm)
run_comm(comm,'Generating spectrum for chip0')

In [ ]:
# shorten ARF and RMF paths
f = fits.open(spec_file)
rmf = f[1].header['RESPFILE']
arf = f[1].header['ANCRFILE']
f.close()

## create symlinks if not already done (uncomment)
#run_comm('ln -s {}'.format(rmf), 'Linking to working directory')
#run_comm('ln -s {}'.format(arf), 'Linking to working directory')

# strip path in keywords to avoid very long values and update header
rmf = os.path.basename(rmf)
arf = os.path.basename(arf)
f = fits.open(spec_file, 'update')
f[1].header['RESPFILE'] = rmf
f[1].header['ANCRFILE'] = arf
f.close()

In [ ]:
# Plot spectrum
Plot.device = '/null'
AllData.clear()
AllModels.clear()

Plot.device = "/xs"
Plot.xAxis="keV"
s1 = Spectrum(spec_file)
Plot("ldata")
Plot.device = '/null'

### <span style="color:blue">$\textbf{Exercise}$</span>:
#### Generate two more SIMPUT files using the *medium* and *low* band, merge them with `simputmerge`, and perform a new WFI simulation.


In [ ]:
# simput file

for band in ['med','low']: 
    sim_file = f'etaCar_{band}.fits'
    img_file = f'etaCar_xray_{band}.fits'
    DeltaE = E_minmax[band][1] - E_minmax[band][0]
    flux = flux_centralsrc_3_8/5.*DeltaE 
    comm = (f'simputfile Simput={sim_file} RA={RA} Dec={Dec} srcFlux={flux} Emin={E_minmax[band][0]} Emax={E_minmax[band][1]} ' + 
            f'Elow={E_lowup[band][0]} Eup={E_lowup[band][1]} XSPECFile={xcm} ImageFile={img_file} clobber=yes')
    #print(comm)
    run_comm(comm, 'Creating simput file for eta car in band {band}')

In [ ]:
# Merge high, med and low  evt files (high obtained in the cells above)
opt = "clobber=yes FetchExtensions=yes"
simput_file_allbands = 'etaCar_allbands.fits'

comm = (f"simputmerge {opt} Infile1=etaCar_low.fits Infile2=etaCar_med.fits Outfile=tmp.fits")
#print(comm)
run_comm(comm, "Merging simputfiles _low_ and _med_")

comm = (f"simputmerge {opt} Infile1=etaCar_high.fits Infile2=tmp.fits Outfile={simput_file_allbands}")
#print(comm)
run_comm(comm, "Merging simputfiles _low_, _med_ and _high")

In [ ]:
# Run the new simulation and merge the 4 chips
base = "etaCar_"
comm = (f'athenawfisim XMLFile0={xml0} XMLFile1={xml1} XMLFile2={xml2} XMLFile3={xml3} RA={RApoint} Dec={DECpoint} ' + 
        f'Prefix={base} Simput={simput_file_allbands} Exposure=1000 clobber=yes')
#print(comm)
run_comm(comm, 'Running simulation for eta car all bands')

# mergin all chips
evt_final_merged = f'{base}allbands_merged_evt.fits'
comm = (f'ftmerge {base}chip0_evt.fits,{base}chip1_evt.fits,{base}chip2_evt.fits,{base}chip3_evt.fits ' + 
        f'{evt_final_merged} clobber=yes')
#print(comm)
run_comm(comm,'Merging 4 event files')


### <span style="color:blue">$\textbf{Exercise (harder)}$</span>:
#### Rather than describing the spectral shapes with constants, describe the overall spectral shape of $\eta$ Car and the Humunculus nebula using an apec model with a temperature of kT = 4 keV, absorbed by NH = 2 $\cdot$ 10$^{22}$ cm$^{−2}$. Use XSPEC to determine the relative fluxes for the three energy bands for which Chandra images are available. Then generate three SIMPUT files, fixing the hard band flux to $10^{−11}$ cgs and the two lower fluxes to values appropriate for the absorbed apec model. Do not forget to set Eup and Elow appropriately. Perform the simulation again

In [ ]:
# xspec file with apec model (https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/XSmodelApec.html) and phabs 
# (https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/node255.html)

xcm = "etaCar_apec.xcm"

AllData.clear()
AllModels.clear()
mcmod = Model("apec * phabs")
mcmod.apec.kT = 4
mcmod.phabs.nH = 2

# Fluxes for the 3 bands
AllModels.calcFlux('2. 7.')
flux_high_nonorm = mcmod.flux[0]

AllModels.calcFlux('0.5 1.2')
flux_low_nonorm = mcmod.flux[0]

AllModels.calcFlux('1.2 2')
flux_med_nonorm = mcmod.flux[0]

flux = dict()
flux["high"] = 1E-11
flux["low"] = (flux_low_nonorm/flux_high_nonorm)*flux["high"]
flux["med"] = (flux_med_nonorm/flux_high_nonorm)*flux["high"]

clobber = True
# If clobber is true and the file exists, it will be removed before creating a new one.
if os.path.exists(xcm):
    if clobber:
        os.remove(xcm)
    else:
        raise Exception(f"Xspec file ({xcm}) already exists: it will not be overwritten") 
Xset.save(xcm)

In [ ]:
base = 'etaCar_apec_'
for band in ["low", "med", "high"]:
    sim_file = f"{base}{band}.fits"
    img_band = band
    if band == "high":
        img_band = "hi"
    comm = (f"simputfile Simput={sim_file} RA={RA} Dec={Dec} srcFlux={flux[band]} " + 
            f"Emin={E_minmax[band][0]} Emax={E_minmax[band][1]} Elow={E_lowup[band][0]} Eup={E_lowup[band][1]} " + 
            f"XSPECFile={xcm} ImageFile=etaCar_xray_{img_band}.fits clobber=yes")
    #print(comm)
    run_comm(comm, f"Creating simput file for apec model and band {band}")

In [ ]:
opt = "clobber=yes FetchExtensions=yes"
simput_file_apec_allbands = 'etaCar_apec_allbands.fits'
comm = (f"simputmerge {opt} Infile1=etaCar_apec_low.fits Infile2=etaCar_apec_med.fits Outfile=tmp.fits")
#print(comm)
run_comm(comm, "Merging simputfiles _low_ and _med_")
comm = (f"simputmerge {opt} Infile1=etaCar_apec_high.fits Infile2=tmp.fits Outfile={simput_file_apec_allbands}")
#print(comm)
run_comm(comm, "Merging simputfiles _low_ and _med_")

In [ ]:
# Run the new simulation and merge the 4 chips
evt_final_merged = f'{base}allbands_merged_evt.fits'

comm = (f'athenawfisim XMLFile0={xml0} XMLFile1={xml1} XMLFile2={xml2} XMLFile3={xml3} RA={RApoint} Dec={DECpoint} ' + 
        f'Prefix={base} Simput={simput_file_apec_allbands} Exposure=10000 clobber=yes')
#print(comm)
run_comm(comm,'Running simulation for eta car')
comm = (f'ftmerge {base}chip0_evt.fits,{base}chip1_evt.fits,{base}chip2_evt.fits,{base}chip3_evt.fits {evt_final_merged} clobber=yes')
#print(comm)
run_comm(comm,'Merging 4 event files')

### <span style="color:blue">$\textbf{Exercise (even harder)}$</span>:
#### For each of the three images, use the Chandra dmcopy tool and an appropriately defined region filter to produce an image file of the humunculus nebula that does not include the central point source and an image file that only includes the point source. Prepare SIMPUT files that model the nebula with a different spectral shape than the point source, merge them with simputmerge, and run the simulation again, now having a very realistic model for the whole source.

(in preparation)